# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey
# Access maps with unique API key
gmaps.configure(api_key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#CSV file with cities data
weather_df = pd.read_csv('/Users/medinai/Desktop/python-api-challenge/WeatherPy/Output/weather_df.csv')
weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santa cruz,17.49,120.43,77.41,87,99,4.85,PH,1602452529
1,1,pacifica,37.61,-122.49,69.01,56,1,17.22,US,1602452446
2,2,impfondo,1.64,18.07,70.23,91,97,2.93,CG,1602452529
3,3,nome,64.50,-165.41,42.80,56,75,6.93,US,1602452529
4,4,thompson,55.74,-97.86,48.20,76,75,4.70,CA,1602452340


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Store Latitude & Longitude into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

humidity = weather_df['Humidity'].astype(float)

max_humidity = humidity.max()

# Create Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity, 
                                 dissipating = False, 
                                 max_intensity = max_humidity,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

new_df=weather_df.loc[(weather_df["Max Temp"] <80) & 
                      (weather_df["Max Temp"]<70) & 
                      (weather_df["Cloudiness"]==0), 
                      :]
new_df.count()
new_final_df = new_df.dropna()    
#new_final_df.count()
new_final_df.reset_index(inplace=True)
del new_final_df['index']
del new_final_df["Unnamed: 0"]
new_final_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rio grande,38.30,21.78,50.00,92,0,3.53,GR,1602452676
1,castro,-24.79,-50.01,60.30,73,0,6.76,BR,1602452676
2,riyadh,24.69,46.72,69.80,26,0,4.70,SA,1602452678
3,puerto ayora,36.59,-6.23,66.20,88,0,2.24,ES,1602452678
4,nushki,29.55,66.02,47.12,45,0,1.81,PK,1602452679


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotels = []

for i in range(len(new_final_df)):
    lat = new_final_df.loc[i]['Lat']
    lng = new_final_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
new_final_df["Hotel Name"] = hotels
hotel_df = new_final_df.dropna(how='any')
hotel_df

/Users/medinai/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,rio grande,38.30,21.78,50.00,92,0,3.53,GR,1602452676,Patras
1,castro,-24.79,-50.01,60.30,73,0,6.76,BR,1602452676,Castro
2,riyadh,24.69,46.72,69.80,26,0,4.70,SA,1602452678,Riyadh
3,puerto ayora,36.59,-6.23,66.20,88,0,2.24,ES,1602452678,El Puerto de Santa María
4,nushki,29.55,66.02,47.12,45,0,1.81,PK,1602452679,Nushki
5,busselton,-33.65,115.33,46.99,83,0,6.22,AU,1602452653,Busselton
6,kruisfontein,-34.00,24.73,53.01,83,0,1.99,ZA,1602452684,Humansdorp
7,lebu,-37.62,-73.65,53.91,62,0,22.10,CL,1602452687,Lebu
8,rawson,-43.30,-65.10,67.21,26,0,1.52,AR,1602452689,Rawson
9,palana,59.12,159.97,32.79,90,0,8.61,RU,1602452689,Palana


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))